# Conda configuration
```bash
conda config --add channels defaults
conda config --add channels bioconda
conda config --add channels conda-forge

conda create -n eztracks --yes python=3
conda activate eztracks
pip install git+https://github.com/deeptools/HiCMatrix.git
conda install -c anaconda freetype
pip install git+https://github.com/deeptools/pyGenomeTracks.git
conda install -c bioconda bedtools
```

---
# Considerations
- Bed files should be presorted ()
- Reserved group names: you cannot use the following names for your groups: `tracks`,`default`

---
- <font color="red">GTF and coordinates (from repeat masker!)</font>

In [125]:
import sys
import os
from os import path,listdir
import subprocess
import argparse
import configparser
from collections import OrderedDict

In [159]:
#~~~~~~~~~~~~~~~~~~~~
def load_config(config_ini):
    if not path.exists(config_ini):
        raise FileNotFoundError("Config file not found. Exiting.")
    config = configparser.ConfigParser()
    config.optionxform = str
    config.read(config_ini)            
    return config

def load_tracks(config_obj):
    config=config_obj
    tracks = OrderedDict()
    for group in config['tracks']:
        P = config['tracks'][group]
        if not path.exists(P):
            raise FileNotFoundError('ERROR: track group',P,'not found.')
        if path.isfile(P):
            tracks[group] = [(P,group)]
        else:            
            files = [path.join(P,x) for x in listdir(P) if x.endswith('.bed') or x.endswith('.bed.gz')]
            if len(files)==0:
                print('# No bed tracks found in %s. Skipping.',file=sys.stderr)
                continue
            tracks[group] = []
            for p in files:                
                p_name = path.basename(p).replace('.bed.gz','').replace('.bed','')                
                tracks[group].append((p,p_name))                
    return tracks
    
def print_config(config_obj):
    for s in config_obj.sections():
        print('[%s]'%s)
        for k in config_obj[s]:
            print('%s = %s'%(k,config_obj[s][k]))

#~~~~~~~~~~~~~~~~~~~~
def check(config_ini):
    errors = 0
    #~~~Check programs
#     try:
#         call = 'echo $CONDA_PREFIX;python -V'
#         call_output = subprocess.check_output(call,shell=True).decode().strip().replace('\n',': ')
#         print('# conda_env:',call_output,file=sys.stderr)
#     except:
#         print('ERROR: Conda environment not found. Exiting.',file=sys.stderr)
#         errors+=1
#     try:
#         call = 'pyGenomeTracks --version'
#         call_output = subprocess.check_output(call,shell=True).decode().strip()
#         print('#',call_output,file=sys.stderr)
#     except:
#         print('ERROR: pyGenomeTracks not found. Exiting.',file=sys.stderr)
#         errors+=1
#     try:
#         call = 'bedtools -version'
#         call_output = subprocess.check_output(call,shell=True).decode().strip()
#         print('#',call_output,file=sys.stderr)
#     except:
#         print('ERROR: bedtools not found. Exiting.',file=sys.stderr)
#         errors+=1
    
    #~~~Check configuration
    if path.exists(config_ini):
        print('# config_file found at',path.abspath(config_ini),file=sys.stderr)
    else:
        print('ERROR: config_file not found. Exiting.',file=sys.stderr)
        return
    config = load_config(config_ini)
    required_params = ['output_path']
    for param in required_params:
        if param not in config['default']:
            print('ERROR: required parameter %s not found in configuration.'%param,file=sys.stderr)
            errors+=1
    
    #~~~Check track files
#     tracks = OrderedDict()
#     for group in config['tracks']:
#         P = config['tracks'][group]
#         if not path.exists(P):
#             print('ERROR: track group',P,'not found.',file=sys.stderr)
#             errors+=1
#             continue
#         print('# Found %s: %s'%(group,P),file=sys.stderr)
#         if path.isfile(P):
#             tracks[group] = [(P,group)]
#         else:
#             tracks[group] = []
#             files = [path.join(P,x) for x in listdir(P) if x.endswith('.bed') or x.endswith('.bed.gz')]
#             for p in files:                
#                 p_name = path.basename(p).replace('.bed.gz','').replace('.bed','')                
#                 tracks[group].append((p,p_name))
#                 print('# Found %s-%s: %s'%(group,p_name,p),file=sys.stderr)    

    #~~~End checkup
    print('# Checkup finalized. %d error(s) detected.'%errors,file=sys.stderr)
    if errors>0:
        print('# Please solve all the configuration errors before calling "prepare" and "draw".',file=sys.stderr)    
    
#~~~~~~~~~~~~~~~~~~~~
def prepare(config_ini):
    pass


#~~~~~~~~~~~~~~~~~~~~
def draw(config_ini):
    pass


def main():
    parser = argparse.ArgumentParser(description='ezTracks. Plot a single GTF annotation followed by grouped bed files.\n\nHugo Guillen, 2020.')
    
    parser.add_argument('action',choices=['check','prepare','draw'],help="""check: check if config file, environment, and input files are correct.
prepare: preprocess input files into output defined in config file.
draw: generate output image into output defined in config file.""")
    parser.add_argument('configfile',help="path to configuration file. For possible options, please refer to the example located at test_data/config.ini")
    args = parser.parse_args()
    if args.action == 'check':
        check(args.configfile)
    elif args.action == 'prepare':
        prepare(args.configfile)
    elif args.action == 'draw':
        draw(args.configfile)
    else:
        parser.print_help()

In [161]:
dummy_args = [sys.argv[0],'--help']
#dummy_args = [sys.argv[0],'check','test_config.ini']
#dummy_args = [sys.argv[0],'prepare','test_config.ini']

sys.argv=dummy_args
main()

usage: --help [-h] {check,prepare,draw} configfile

ezTracks. Plot a single GTF annotation followed by grouped bed files. Hugo
Guillen, 2020.

positional arguments:
  {check,prepare,draw}  check: check if config file, environment, and input
                        files are correct. prepare: preprocess input files
                        into output defined in config file. draw: generate
                        output image into output defined in config file.
  configfile            path to configuration file. For possible options,
                        please refer to the example located at
                        test_data/config.ini

optional arguments:
  -h, --help            show this help message and exit


SystemExit: 0

/home/hugo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
